# Условие задачи

Кто там? Предскажите, кто вошел в здание по времени и турникету. Но теперь легче: мы знаем, что "след" через турникеты принадлежит "Х". Кто этот "Х" предстоит вычислить по данным, которые нам известны. А ID юзеров известны по обучающей выборке (кроме нескольких новых!).
Чтобы попасть в здание, нужно пройти через турникет. Чтобы открыть парковку, нужно открыть шлагбаум. Чтобы попасть на этаж, нужно приложить “таблетку”. Все это фиксируется - кто, в какую дату, в какое время.
Сможем ли мы выучить кому конкретно принадлежит след прохода через турникеты? Теперь мы знаем, что это был кто-то, кто уже был. (Правда, есть и несколько - не больше 10% - новых).
“8 утра понедельник, турникет 4? Директор.” “11 утра суббота? Гриша. Но Гриша в последний день месяца никогда не приходит.”  Какие есть паттерны в настоящих данных?

В тестовой выборке id посетителей заменены на слова, которые встречались в курсе "Линейные модели": aucroc, binary, blue, categorical и т.п.
Ваша задача в этом раунде составить таблицу вида:
```
user_word	preds
aucroc	49
binary	12
blue	55
categorical	-999
coefficient	15
```
где user_word - index, preds - колонка с соответствующими id. Таблицу нужно запомнить как csv (см. пример в секции Data). -999 – id нового посетителя, которого раньше не было.
Каждому слову сопоставлен вес, который зависит от того, насколько трудно предсказать id. Баллы набираются как взвешенная по весам сумма правильных ответов. Чем сложнее вы отгадали слово, тем больше баллов.

Отгадали все? Забирайте 560 баллов и становитесь победителем.

Используем деревья и ансамбли.

In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [27]:
input_train_df = pd.read_csv('train.csv', index_col=0)
input_train_df.ts = pd.to_datetime(input_train_df.ts)
input_test_df = pd.read_csv('test.csv', index_col = 0)
input_test_df.ts = pd.to_datetime(input_test_df.ts)

In [28]:
input_train_df

,user_id,ts,gate_id
0,18,2022-07-29 09:08:54,7
1,18,2022-07-29 09:09:54,9
2,18,2022-07-29 09:09:54,9
3,18,2022-07-29 09:10:06,5
4,18,2022-07-29 09:10:08,5
...,...,...,...
37513,6,2022-12-31 20:38:56,11
37514,6,2022-12-31 20:39:22,6
37515,6,2022-12-31 20:39:23,6
37516,6,2022-12-31 20:39:31,9


In [29]:
input_test_df

,ts,gate_id,user_word
37518,2023-01-03 08:21:00,9,gini
37519,2023-01-03 08:21:00,9,gini
37520,2023-01-03 08:21:18,5,gini
37521,2023-01-03 08:21:19,5,gini
37522,2023-01-03 08:21:39,10,gini
...,...,...,...
44638,2023-02-24 19:43:36,11,collinear
44639,2023-02-24 19:44:00,4,collinear
44640,2023-02-24 19:44:01,4,collinear
44641,2023-02-24 19:44:09,9,collinear


# Работа с данными

Разобъём данные на несколько логичных стоблцов. Время в юникс, номер дня в неделе, рабочий ли день

In [59]:
train_df = pd.DataFrame(input_train_df)
test_df = pd.DataFrame(input_test_df)
train_df['Time'] = train_df['ts'].apply(lambda x: x.timestamp())
train_df['DayPosition'] = train_df['ts'].dt.dayofweek
train_df['IsWorkDay'] = train_df['ts'].dt.weekday < 5
train_df['IsWorkDay'] = train_df['IsWorkDay'].astype(int)
#
test_df['Time'] = test_df['ts'].apply(lambda x: x.timestamp())
test_df['DayPosition'] = test_df['ts'].dt.dayofweek
test_df['IsWorkDay'] = test_df['ts'].dt.weekday < 5
test_df['IsWorkDay'] = test_df['IsWorkDay'].astype(int)

Проверим для первого пользователя всё ли у нас хорошо.

In [60]:
test_df

,ts,gate_id,user_word,Time,DayPosition,IsWorkDay
37518,2023-01-03 08:21:00,9,gini,1.672734e+09,1,1
37519,2023-01-03 08:21:00,9,gini,1.672734e+09,1,1
37520,2023-01-03 08:21:18,5,gini,1.672734e+09,1,1
37521,2023-01-03 08:21:19,5,gini,1.672734e+09,1,1
37522,2023-01-03 08:21:39,10,gini,1.672734e+09,1,1
...,...,...,...,...,...,...
44638,2023-02-24 19:43:36,11,collinear,1.677268e+09,4,1
44639,2023-02-24 19:44:00,4,collinear,1.677268e+09,4,1
44640,2023-02-24 19:44:01,4,collinear,1.677268e+09,4,1
44641,2023-02-24 19:44:09,9,collinear,1.677268e+09,4,1


In [61]:
train_df

,user_id,ts,gate_id,Time,DayPosition,IsWorkDay
0,18,2022-07-29 09:08:54,7,1.659086e+09,4,1
1,18,2022-07-29 09:09:54,9,1.659086e+09,4,1
2,18,2022-07-29 09:09:54,9,1.659086e+09,4,1
3,18,2022-07-29 09:10:06,5,1.659086e+09,4,1
4,18,2022-07-29 09:10:08,5,1.659086e+09,4,1
...,...,...,...,...,...,...
37513,6,2022-12-31 20:38:56,11,1.672519e+09,5,0
37514,6,2022-12-31 20:39:22,6,1.672519e+09,5,0
37515,6,2022-12-31 20:39:23,6,1.672519e+09,5,0
37516,6,2022-12-31 20:39:31,9,1.672519e+09,5,0


In [62]:
train_df.loc[train_df['user_id']== 3]

,user_id,ts,gate_id,Time,DayPosition,IsWorkDay
20,3,2022-07-29 09:40:40,7,1.659088e+09,4,1
21,3,2022-07-29 09:42:49,9,1.659088e+09,4,1
22,3,2022-07-29 09:42:49,9,1.659088e+09,4,1
23,3,2022-07-29 09:43:01,5,1.659088e+09,4,1
24,3,2022-07-29 09:43:03,5,1.659088e+09,4,1
...,...,...,...,...,...,...
37243,3,2022-12-30 09:17:59,5,1.672392e+09,4,1
37244,3,2022-12-30 09:18:26,10,1.672392e+09,4,1
37445,3,2022-12-30 18:46:40,11,1.672426e+09,4,1
37446,3,2022-12-30 18:47:08,4,1.672426e+09,4,1


In [63]:
train_df.loc[train_df['user_id']== 3].head(20)

,user_id,ts,gate_id,Time,DayPosition,IsWorkDay
20,3,2022-07-29 09:40:40,7,1.659088e+09,4,1
21,3,2022-07-29 09:42:49,9,1.659088e+09,4,1
22,3,2022-07-29 09:42:49,9,1.659088e+09,4,1
23,3,2022-07-29 09:43:01,5,1.659088e+09,4,1
24,3,2022-07-29 09:43:03,5,1.659088e+09,4,1
25,3,2022-07-29 09:43:29,10,1.659088e+09,4,1
197,3,2022-07-29 12:57:17,11,1.659099e+09,4,1
198,3,2022-07-29 12:57:42,4,1.659099e+09,4,1
199,3,2022-07-29 12:57:44,4,1.659099e+09,4,1
253,3,2022-07-29 14:00:48,7,1.659103e+09,4,1


Что мы видим? 
1) У нас есть дубликаты записей, которые нужно удалить; 
2) Если мы предполагаем что за пользователем только один человек и у нас валидные данные, то у нас получается, что 3-ий пользователь вошёл в 9:40 и в конце вечера снова вошёл в здание и не ушёл, что не совсем реалистично. 
А значит делать фичу на вход/выход делать бессмысленно. А может нужно просто выкинуть первый день?
3) Стоит попробовать всё-таки выкинуть первый день и добавить фичу зашёл/вышел.

## Удаляем дубликаты

In [64]:
train_df = train_df.drop_duplicates(subset=['ts', 'user_id'])
test_df = test_df.drop_duplicates(subset=['ts', 'user_word'])

In [66]:
train_df

,user_id,ts,gate_id,Time,DayPosition,IsWorkDay
0,18,2022-07-29 09:08:54,7,1.659086e+09,4,1
1,18,2022-07-29 09:09:54,9,1.659086e+09,4,1
3,18,2022-07-29 09:10:06,5,1.659086e+09,4,1
4,18,2022-07-29 09:10:08,5,1.659086e+09,4,1
5,18,2022-07-29 09:10:34,10,1.659086e+09,4,1
...,...,...,...,...,...,...
37512,6,2022-12-31 17:21:19,10,1.672507e+09,5,0
37513,6,2022-12-31 20:38:56,11,1.672519e+09,5,0
37514,6,2022-12-31 20:39:22,6,1.672519e+09,5,0
37515,6,2022-12-31 20:39:23,6,1.672519e+09,5,0


In [67]:
train_df.loc[train_df['user_id']== 3]

,user_id,ts,gate_id,Time,DayPosition,IsWorkDay
20,3,2022-07-29 09:40:40,7,1.659088e+09,4,1
21,3,2022-07-29 09:42:49,9,1.659088e+09,4,1
23,3,2022-07-29 09:43:01,5,1.659088e+09,4,1
24,3,2022-07-29 09:43:03,5,1.659088e+09,4,1
25,3,2022-07-29 09:43:29,10,1.659088e+09,4,1
...,...,...,...,...,...,...
37243,3,2022-12-30 09:17:59,5,1.672392e+09,4,1
37244,3,2022-12-30 09:18:26,10,1.672392e+09,4,1
37445,3,2022-12-30 18:46:40,11,1.672426e+09,4,1
37446,3,2022-12-30 18:47:08,4,1.672426e+09,4,1


In [68]:
test_df

,ts,gate_id,user_word,Time,DayPosition,IsWorkDay
37518,2023-01-03 08:21:00,9,gini,1.672734e+09,1,1
37520,2023-01-03 08:21:18,5,gini,1.672734e+09,1,1
37521,2023-01-03 08:21:19,5,gini,1.672734e+09,1,1
37522,2023-01-03 08:21:39,10,gini,1.672734e+09,1,1
37523,2023-01-03 08:32:49,15,epsilon,1.672735e+09,1,1
...,...,...,...,...,...,...
44637,2023-02-24 17:08:57,10,collinear,1.677259e+09,4,1
44638,2023-02-24 19:43:36,11,collinear,1.677268e+09,4,1
44639,2023-02-24 19:44:00,4,collinear,1.677268e+09,4,1
44640,2023-02-24 19:44:01,4,collinear,1.677268e+09,4,1


## Убираем первый день из train

In [92]:
target_date = pd.to_datetime('2022-07-29')

# Установите фильтр для удаления данных за указанный день
filter_condition = train_df['ts'].dt.date != target_date.date()

# Примените фильтр к DataFrame
train_df = train_df[filter_condition]

In [93]:
train_df

,user_id,ts,gate_id,Time,DayPosition,IsWorkDay
505,29,2022-07-30 09:53:31,7,1.659175e+09,5,0
506,29,2022-07-30 09:55:15,9,1.659175e+09,5,0
508,29,2022-07-30 09:55:24,5,1.659175e+09,5,0
509,29,2022-07-30 09:55:26,5,1.659175e+09,5,0
510,29,2022-07-30 09:55:54,10,1.659175e+09,5,0
...,...,...,...,...,...,...
37512,6,2022-12-31 17:21:19,10,1.672507e+09,5,0
37513,6,2022-12-31 20:38:56,11,1.672519e+09,5,0
37514,6,2022-12-31 20:39:22,6,1.672519e+09,5,0
37515,6,2022-12-31 20:39:23,6,1.672519e+09,5,0


Добавляем в train столбец вошёл/вышел

In [103]:
train_df['isEnter'] = train_df.groupby('user_id').cumcount() % 2 == 0

C:\Users\Skiv2\AppData\Local\Temp\ipykernel_28044\1601702820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['isEnter'] = train_df.groupby('user_id').cumcount() % 2 == 0


In [107]:
train_df

,user_id,ts,gate_id,Time,DayPosition,IsWorkDay,isEnter
505,29,2022-07-30 09:53:31,7,1.659175e+09,5,0,True
506,29,2022-07-30 09:55:15,9,1.659175e+09,5,0,False
508,29,2022-07-30 09:55:24,5,1.659175e+09,5,0,True
509,29,2022-07-30 09:55:26,5,1.659175e+09,5,0,False
510,29,2022-07-30 09:55:54,10,1.659175e+09,5,0,True
...,...,...,...,...,...,...,...
37512,6,2022-12-31 17:21:19,10,1.672507e+09,5,0,False
37513,6,2022-12-31 20:38:56,11,1.672519e+09,5,0,True
37514,6,2022-12-31 20:39:22,6,1.672519e+09,5,0,False
37515,6,2022-12-31 20:39:23,6,1.672519e+09,5,0,True


In [109]:
train_df.loc[train_df['user_id']== 29]

,user_id,ts,gate_id,Time,DayPosition,IsWorkDay,isEnter
505,29,2022-07-30 09:53:31,7,1.659175e+09,5,0,True
506,29,2022-07-30 09:55:15,9,1.659175e+09,5,0,False
508,29,2022-07-30 09:55:24,5,1.659175e+09,5,0,True
509,29,2022-07-30 09:55:26,5,1.659175e+09,5,0,False
510,29,2022-07-30 09:55:54,10,1.659175e+09,5,0,True
...,...,...,...,...,...,...,...
37438,29,2022-12-30 17:27:00,10,1.672421e+09,4,1,True
37500,29,2022-12-30 20:34:29,11,1.672432e+09,4,1,False
37501,29,2022-12-30 20:34:56,4,1.672432e+09,4,1,True
37503,29,2022-12-30 20:34:58,4,1.672432e+09,4,1,False


In [111]:
test_df['isEnter'] = test_df.groupby('user_word').cumcount() % 2 == 0

C:\Users\Skiv2\AppData\Local\Temp\ipykernel_28044\380260690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['isEnter'] = test_df.groupby('user_word').cumcount() % 2 == 0


In [112]:
test_df

,ts,gate_id,user_word,Time,DayPosition,IsWorkDay,isEnter
37518,2023-01-03 08:21:00,9,gini,1.672734e+09,1,1,True
37520,2023-01-03 08:21:18,5,gini,1.672734e+09,1,1,False
37521,2023-01-03 08:21:19,5,gini,1.672734e+09,1,1,True
37522,2023-01-03 08:21:39,10,gini,1.672734e+09,1,1,False
37523,2023-01-03 08:32:49,15,epsilon,1.672735e+09,1,1,True
...,...,...,...,...,...,...,...
44637,2023-02-24 17:08:57,10,collinear,1.677259e+09,4,1,False
44638,2023-02-24 19:43:36,11,collinear,1.677268e+09,4,1,True
44639,2023-02-24 19:44:00,4,collinear,1.677268e+09,4,1,False
44640,2023-02-24 19:44:01,4,collinear,1.677268e+09,4,1,True


In [114]:
test_df.loc[test_df['user_word'] == 'gini']

,ts,gate_id,user_word,Time,DayPosition,IsWorkDay,isEnter
37518,2023-01-03 08:21:00,9,gini,1.672734e+09,1,1,True
37520,2023-01-03 08:21:18,5,gini,1.672734e+09,1,1,False
37521,2023-01-03 08:21:19,5,gini,1.672734e+09,1,1,True
37522,2023-01-03 08:21:39,10,gini,1.672734e+09,1,1,False
37563,2023-01-03 10:47:32,11,gini,1.672743e+09,1,1,True
...,...,...,...,...,...,...,...
44565,2023-02-23 08:06:18,10,gini,1.677140e+09,3,1,True
44575,2023-02-23 17:56:59,11,gini,1.677175e+09,3,1,False
44576,2023-02-23 17:57:16,4,gini,1.677175e+09,3,1,True
44577,2023-02-23 17:57:18,4,gini,1.677175e+09,3,1,False


# Прогнозируем

Необходимо построить деревья и ансамбли деревьев и вывести лучшую модель